# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
import tensorflow as tf
import pandas as pd
import pickle
warnings.filterwarnings('ignore')

2024-06-18 10:50:49.092456: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 10:50:49.848056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose

# 1. Load Model

In [3]:
# path to the model
dir_model = "../model/xyz/"

# you can try different model
# dir_model = "../Model/xyz_hand/"
# dir_model = "../Model/xy/"
# dir_model = "../Model/xy_hand/"

loaded_model = tf.saved_model.load(dir_model)
inferer = loaded_model.signatures["serving_default"]

2024-06-18 10:50:52.816436: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 10:50:52.854016: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 10:50:52.854079: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 10:50:52.857758: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-18 10:50:52.857830: I external/local_xla/xla/stream_executor

# 2. Setup OpenCV and MediaPipe to Predict

In [ ]:
dir_video_test = ('../../data/all videos/test9.mp4')
cap = cv2.VideoCapture(dir_video_test)
current_stage = ''

#initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    # stream the video
    while cap.isOpened():
        ret, frame = cap.read()
        
        # recolor feed
        # image = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # make detection
        results = pose.process(image)
        
        # recolor image back to BGR for rendering
        image.flags.writeable = False
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # try to predict
        try:
            # model xyz
            row = np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]).flatten()
            X = pd.DataFrame([row])
            ROM_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            ROM_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xyz_hand
            # row = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # X = pd.DataFrame([row])
            #ROM_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            #ROM_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xy
            # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[]]).flatten()
            # X = pd.DataFrame([row])
            #ROM_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            #ROM_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xy_hand
            # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # X = pd.DataFrame([row])
            #ROM_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            #ROM_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            print(ROM_class,ROM_prob)

            if ROM_class == 0 and ROM_prob >= 0.9 :
                current_stage = 'NOT'
            elif ROM_class == 1 and ROM_prob >= 0.9 :
                current_stage = 'YES!'

            # get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

            #Display class
            cv2.putText(image, 'ROM', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

            #Display Prob
            cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(ROM_prob, 2)), (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
            
        except Exception as e:
            print(e)
            
        # Stream vid resultq
        cv2.imshow("Raw Cam Feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

W0000 00:00:1718682878.759734    9244 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718682878.779520    9244 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0xa8d15e0) is not the object's thread (0x9094cd0).
Cannot move to target thread (0xa8d15e0)

QObject::moveToThread: Current thread (0xa8d15e0) is not the object's thread (0x9094cd0).
Cannot move to target thread (0xa8d15e0)

QObject::moveToThread: Current thread (0xa8d15e0) is not the object's thread (0x9094cd0).
Cannot move to target thread (0xa8d15e0)

QObject::moveToThread: Current thread (0xa8d15e0) is not the object's thread (0x9094cd0).
Cannot move to target thread (0xa8d15e0)

QObject::moveToThread: Current thread (0xa8d15e0) is not the object's thread (0x9094cd0).
Cannot move to tar

1 0.98529774
1 0.9937215
1 0.9959416
1 0.99637765
1 0.99711007
1 0.9976593
1 0.9979894
1 0.998304
1 0.9988601
1 0.99915695
1 0.9993724
1 0.99937433
1 0.9993625
1 0.9993981
1 0.9963116
1 0.9921407
1 0.9832456
1 0.96895516
1 0.8827682
0 0.68850136
0 0.8877637
0 0.9291917
0 0.8346482
1 0.7873445
1 0.8442621
1 0.7134524
0 0.6855829
0 0.6119005
1 0.89742553
1 0.99533314
1 0.9998846
1 0.99999714
1 0.9999994
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 0.9999999
1 0.9999982
1 0.9999081
1 0.9991949
1 0.995974
1 0.98775977
1 0.9822781
1 0.8022834
1 0.6503284
1 0.6310856
0 0.8317435
0 0.9348532
0 0.88049406
0 0.91099846
0 0.97265846
0 0.99038976
0 0.98447096
0 0.9624207
0 0.93629587
0 0.85350066
0 0.8255457
0 0.75790846
0 0.70863974
0 0.5601959
0 0.51328945
1 0.7483979
1 0.8715594
1 0.92232627
1 0.95385826
1 0.94514275
1 0.94539446
1 0.9642995
1 0.9855304
1 0.98654395
1 0.9892811
1 0.99121726
1 0.9912164
1 0.99285156
1 0.99362826
1 0.99858737
